In [1]:
import pandas as pd
import clickhouse_connect
import os

In [ ]:
client = clickhouse_connect.get_client(
    host='clickhouse',
    port=8123,  # HTTP interface
    username='your_user',
    password='your_password'
)

In [3]:
client.command('DROP DATABASE IF EXISTS superficie_bd')

In [4]:
client.command("CREATE DATABASE IF NOT EXISTS superficie_bd") #creación de base de datos para bosques y cultivos 

In [5]:
print(client.query("SHOW DATABASES").result_rows)

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('superficie_bd',), ('system',)]


In [6]:
# Read CSV files
csv_path_1 = '/usr/workspace/media/00_aprovechamiento_forestal.csv'
data_1 = pd.read_csv(csv_path_1)

csv_path_2 = '/usr/workspace/media/00_cultivos.csv'
data_2 = pd.read_csv(csv_path_2, na_values=['*'])

In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 46 columns):
 #   Column                                                                              Non-Null Count  Dtype  
---  ------                                                                              --------------  -----  
 0   entidad_federativa                                                                  32 non-null     object 
 1   poblacion_2020                                                                      32 non-null     int64  
 2   superficie_entidad_hectareas                                                        32 non-null     float64
 3   total_superficie_forestal_ha                                                        32 non-null     float64
 4   superficie_total_de_bosque_ha                                                       32 non-null     float64
 5   superficie_total_de_selva_ha                                                        32 non-null     f

In [8]:
data_tuples_1 = [tuple(row) for row in data_1.values]
#data_tuples_1

In [9]:
# CREATE TABLE statement
create_table_1 = """
CREATE TABLE IF NOT EXISTS superficie_bd.superficie_forestal (
    entidad_federativa String,
    poblacion_2020 UInt32,
    superficie_entidad_hectareas Float64,
    total_superficie_forestal_ha Float64,
    superficie_total_de_bosque_ha Float64,
    superficie_total_de_selva_ha Float64,
    superficie_total_de_manglar_ha Float64,
    superficie_otras_asociaciones_ha Float64,
    superficie_subtotal_arbolado_ha Float64,
    superficie_total_matorral_xerofilo_ha Float64,
    superficie_total_otras_areas_forestales_ha Float64,
    area_no_forestal_agricultura_de_humedad_ha Float64,
    area_no_forestal_agricultura_de_riego_ha Float64,
    area_no_forestal_agricultura_de_temporal_ha Float64,
    area_no_forestal_cuerpo_de_agua_ha Float64,
    area_no_forestal_acuicola_ha Float64,
    area_no_forestal_pastizal_cultivado_ha Float64,
    area_no_forestal_pastizal_inducido_ha Float64,
    area_no_forestal_sin_vegetacion_aparente_ha Float64,
    area_no_forestal_desprovisto_de_vegetacion_ha Float64,
    area_no_forestal_asentamientos_humanos_ha Float64,
    total_unidades_de_produccion_forestal UInt32,
    total_hectareas_produccion_forestal Float64,
    total_unidades_con_superficie_reforestada UInt32,
    unidades_reforestadas_para_restauracion_del_entorno_y_proteccion_del_suelo_y_agua UInt32,
    total_superficie_reforestada_hectareas Float64,
    superficie_reforestada_para_restauracion_del_entorno_y_proteccion_del_suelo_y_agua Float64,
    total_unidades_de_produccion_con_superficie_deforestada UInt32,
    total_superficie_deforestada_hectareas Float64,
    superficie_deforestada_por_causas_naturales Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_agricola UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_agricola Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_pastoreo_de_ganado UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_pastoreo_de_ganado Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_habitacional UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_habitacional Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_comercio UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_comercio Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_industria UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_industria Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_servicios UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_servicios Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_forestal UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_forestal Float64,
    unidades_deforestadas_cambio_de_uso_de_suelo_otros_usos UInt32,
    superficie_deforestada_cambio_de_uso_de_suelo_otros_usos Float64
) ENGINE = MergeTree()
ORDER BY entidad_federativa
"""

In [10]:
# Execute the CREATE TABLE statement
try:
    client.command(create_table_1)
    print("Table 'superficie_forestal' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")

Table 'superficie_forestal' created successfully!


In [11]:
# Insert the DataFrame data
try:
    client.insert('superficie_bd.superficie_forestal', data_tuples_1)
    print(f"Successfully inserted {len(data_1)} rows into superficie_forestal table")
except Exception as e:
    print(f"Error inserting data: {e}")

Successfully inserted 32 rows into superficie_forestal table


In [12]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42951 entries, 0 to 42950
Data columns (total 18 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   clave_entidad_federativa                 42951 non-null  object 
 1   entidad_federativa                       42951 non-null  object 
 2   municipio                                42951 non-null  object 
 3   tipo_cultivo                             42951 non-null  object 
 4   cultivo                                  42951 non-null  object 
 5   total_unidades_produccion_activas        30106 non-null  float64
 6   total_unidades_cultivo_cielo_abierto     30106 non-null  float64
 7   total_superficie_sembrada_hectareas      42951 non-null  float64
 8   total_superficie_cosechada_hectareas     42951 non-null  float64
 9   total_produccion_toneladas_cosechadas    42951 non-null  float64
 10  unidades_produccion_temporal             30106

In [13]:
data_tuples_2 = [tuple(row) for row in data_2.values]

In [14]:
# CREATE TABLE statement
create_table_2 = """
CREATE TABLE IF NOT EXISTS superficie_bd.cultivos (
    clave_entidad_federativa String,
    entidad_federativa String,
    municipio String,
    tipo_cultivo String,
    cultivo String,
    total_unidades_produccion_activas Float64,
    total_unidades_cultivo_cielo_abierto Float64,
    total_superficie_sembrada_hectareas Float64,
    total_superficie_cosechada_hectareas Float64,
    total_produccion_toneladas_cosechadas Float64,
    unidades_produccion_temporal Float64,
    superficie_sembrada_temporal_hectareas Float64,
    superficie_cosechada_temporal_hectareas Float64,
    produccion_temporal_toneladas Float64,
    unidades_produccion_riego Float64,
    superficie_sembrada_riego_hectareas Float64,
    superficie_cosechada_riego_hectareas Float64,
    produccion_riego_toneladas Float64
) ENGINE = MergeTree()
ORDER BY (entidad_federativa, cultivo, municipio, tipo_cultivo)
"""

In [15]:
# Execute the CREATE TABLE statement
try:
    client.command(create_table_2)
    print("Table 'cultivos' created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")

Table 'cultivos' created successfully!


In [16]:
# Insert the DataFrame data
try:
    client.insert('superficie_bd.cultivos', data_tuples_2)
    print(f"Successfully inserted {len(data_2)} rows into cultivos table")
except Exception as e:
    print(f"Error inserting data: {e}")

Successfully inserted 42951 rows into cultivos table


In [17]:
result = client.query('''SELECT entidad_federativa, poblacion_2020, total_superficie_forestal_ha
                        FROM superficie_bd.superficie_forestal
                        ORDER BY total_superficie_forestal_ha DESC''')
result.result_rows

[('Chihuahua', 3741869, 21610804.96),
 ('Sonora', 2944840, 14947828.6),
 ('Coahuila de Zaragoza', 3146771, 13742860.65),
 ('Durango', 1832650, 10618110.32),
 ('Baja California Sur', 798447, 6818426.016),
 ('Oaxaca', 4132148, 6692729.994),
 ('Baja California', 8769020, 6299637.143),
 ('Zacatecas', 1622138, 5224235.25),
 ('Jalisco', 8348151, 4546511.5),
 ('Guerrero', 3540685, 4339165.146),
 ('Campeche', 928363, 4330835.342),
 ('San Luis Potosí', 2822255, 4248261.253),
 ('Nuevo León', 5784442, 4196112.14),
 ('Quintana Roo', 1857985, 3877252.915),
 ('Tamaulipas', 3527735, 3823023.045),
 ('Chiapas', 5543828, 3585555.074),
 ('Michoacán de Ocampo', 4748846, 3519015.468),
 ('Sinaloa', 3026943, 3205202.299),
 ('Yucatán', 2320898, 2694374.543),
 ('Nayarit', 1235456, 1918333.334),
 ('Puebla', 6583278, 1573554.131),
 ('Veracruz de Ignacio de la Llave', 8062579, 1471181.313),
 ('Guanajuato', 6166934, 1262179.569),
 ('Hidalgo', 3082841, 962134.053),
 ('Tabasco', 2402598, 817154.1303),
 ('México', 16